## Підготовка та Аналіз даних
___
# **Лабораторна робота №2**
___
#### Наука про дані: підготовчий етап
___

**Мета роботи:**  
Ознайомитися з основними кроками по роботі з даними – workflow від постановки задачі до написання пояснювальної записки, зрозуміти постановку задачі та природу даних, над якими виконується аналітичні операції.

**Основні поняття:**

- Сирі дані (raw data)
- Підготовка даних (data preparation)
***
*Роботу виконав:*
ФБ-35 Пятілєткін Данііл

Import libs:

In [ ]:
from datetime import datetime
import requests
import os
import pandas as pd
import matplotlib.pyplot as plt

print("Libs imported")

Libs imported


Для кожної із адміністративних одиниць України завантажити тестові
структуровані файли, що містять значення VHI-індексу. Ця процедура має бути
автоматизована, параметром процедури має бути індекс (номер) області. При
зберіганні файлу до його імені потрібно додати дату та час завантаження.

Передбачити повторні запуски скрипту, довантаження нових даних та колізію
даних;

In [53]:
def download_vhi_data(province_id):
    url = f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={province_id}&year1=1981&year2=2024&type=Mean"
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    folder = "Data"
    if not os.path.exists(folder):
        os.makedirs(folder)
    filename = os.path.join(folder, f"vhi_{province_id}_{timestamp}.csv")
    
    response = requests.get(url)
    with open(filename, 'wb') as f:
        f.write(response.content)
    print(f"Data for {province_id} saved to {filename}")

for province_id in range(1, 28):
    download_vhi_data(province_id)

Data for 1 saved to Data\vhi_1_20250311_135112.csv
Data for 2 saved to Data\vhi_2_20250311_135113.csv
Data for 3 saved to Data\vhi_3_20250311_135114.csv
Data for 4 saved to Data\vhi_4_20250311_135115.csv
Data for 5 saved to Data\vhi_5_20250311_135116.csv
Data for 6 saved to Data\vhi_6_20250311_135118.csv
Data for 7 saved to Data\vhi_7_20250311_135118.csv
Data for 8 saved to Data\vhi_8_20250311_135119.csv
Data for 9 saved to Data\vhi_9_20250311_135120.csv
Data for 10 saved to Data\vhi_10_20250311_135121.csv
Data for 11 saved to Data\vhi_11_20250311_135122.csv
Data for 12 saved to Data\vhi_12_20250311_135123.csv
Data for 13 saved to Data\vhi_13_20250311_135124.csv
Data for 14 saved to Data\vhi_14_20250311_135124.csv
Data for 15 saved to Data\vhi_15_20250311_135125.csv
Data for 16 saved to Data\vhi_16_20250311_135126.csv
Data for 17 saved to Data\vhi_17_20250311_135127.csv
Data for 18 saved to Data\vhi_18_20250311_135128.csv
Data for 19 saved to Data\vhi_19_20250311_135129.csv
Data for 20

Зчитати завантажені текстові файли у фрейм
(https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) (детальніше
про роботу із фреймами буде розказано у подальших лабораторних роботах).
Імена стовбців фрейму мають бути змістовними та легкими для сприйняття (не
повинно бути спеціалізованих символів, пробілів тощо). Ця задача має бути
реалізована у вигляді окремої процедури, яка на вхід приймає шлях до
директорії, в якій зберігаються файли;

In [54]:
def read_data(dir='Data'):
    headers = ['year', 'week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI','smth']
    all_data = pd.DataFrame()
    for filename in os.listdir(dir):
        if not filename.startswith("vhi_") or not filename.endswith(".csv"):
            continue
        try:
            province_id = filename.split("_")[1]
            file_path = os.path.join(dir, filename)
            df = pd.read_csv(file_path, header=1, names=headers)
            df = df[df['VHI'] != -1]
            df['year'] = df['year'].str.replace(r'<.*?>', '', regex=True)
            df = df[~df['year'].str.contains('</pre></tt>')]
            df['province_id'] = province_id
            df.drop('smth', axis=1, inplace=True)
            all_data = pd.concat([all_data, df]).drop_duplicates().reset_index(drop=True)
        except Exception as err:
            print(f"Error {filename}: {err}")
        all_data = pd.concat([all_data, df]).drop_duplicates().reset_index(drop=True)
    all_data['province_id'] = all_data['province_id'].astype(int)
    all_data = all_data.sort_values(by=['province_id']).reset_index(drop=True)
    all_data.to_csv('out.csv')
    return all_data

df = read_data()
print(df)
    

       year  week    SMN     SMT    VCI    TCI    VHI  province_id
0      1982   1.0  0.053  260.31  45.01  39.46  42.23            1
1      2011  11.0  0.117  277.33  51.92  27.97  39.95            1
2      2011  10.0  0.102  274.58  50.96  31.36  41.16            1
3      2011   9.0  0.086  271.31  47.97  33.76  40.87            1
4      2011   8.0  0.071  268.21  43.15  36.55  39.85            1
...     ...   ...    ...     ...    ...    ...    ...          ...
59044  1996  34.0  0.386  291.77  46.14  73.42  59.78           27
59045  1996  35.0  0.371  290.30  45.98  77.97  61.97           27
59046  1996  36.0  0.353  288.11  46.79  86.58  66.69           27
59047  1996  30.0  0.392  293.19  32.80  73.41  53.10           27
59048  1982   1.0  0.083  261.70  57.78  41.22  49.50           27

[59049 rows x 8 columns]


Реалізувати окрему процедуру, яка змінить індекси областей, які використані на
порталі NOAA (за англійською абеткою) на наступні, за українською (виключно
старі індекси на нові):

In [57]:
region_map = {
        24: "Вінницька", 25: "Волинська", 5: "Дніпропетровська", 6: "Донецька",
        27: "Житомирська", 23: "Закарпатська", 26: "Запорізька", 7: "Івано-Франківська",
        11: "Київська", 12:"Київ", 13: "Кіровоградська", 14: "Луганська", 15: "Львівська",
        16: "Миколаївська", 17: "Одеська", 18: "Полтавська", 19: "Рівненська", 20: "Севастополь",
        21: "Сумська", 22: "Тернопільська", 8: "Харківська", 9: "Херсонська",
        10: "Хмельницька", 1: "Черкаська", 2: "Чернігівська", 3: "Чернівецька",
        4: "Республіка Крим"
    }

ordered_regions = [
    "Вінницька", "Волинська", "Дніпропетровська", "Донецька", "Житомирська",
    "Закарпатська", "Запорізька", "Івано-Франківська", "Київська", "Київ",
    "Кіровоградська", "Луганська", "Львівська", "Миколаївська", "Одеська", "Полтавська",
    "Рівненська", "Сумська", "Тернопільська", "Харківська", "Херсонська", "Хмельницька",
    "Черкаська", "Чернівецька", "Чернігівська", "Севастополь", "Республіка Крим"
]

def map_region_names(df, id_column="province_id"):
    df["region_name"] = df[id_column].map(region_map)
    df["sort_order"] = df["region_name"].apply(lambda x: ordered_regions.index(x) if x in ordered_regions else len(ordered_regions))
    df = df.sort_values(by=["sort_order"]).drop(columns=["sort_order"]).reset_index(drop=True)
    
    return df

df = map_region_names(df)
df.to_csv("out.csv")
print(df)

       year  week    SMN     SMT    VCI    TCI    VHI  province_id  \
0             NaN    NaN     NaN    NaN    NaN    NaN           24   
1      2010  48.0  0.099  268.36  59.27  39.63  49.45           24   
2      2011   1.0  0.042  256.07  30.13  54.29  42.21           24   
3      2003  32.0  0.427  294.98  69.73  68.78  69.25           24   
4      2010  49.0  0.079  264.23  53.74  49.43  51.59           24   
...     ...   ...    ...     ...    ...    ...    ...          ...   
59044  1996  22.0  0.376  301.64  55.41  16.39  35.90            4   
59045  1996  21.0  0.372  300.86  52.23  14.62  33.42            4   
59046  1996  35.0  0.245  294.89  37.74  89.03  63.38            4   
59047  1996  26.0  0.323  306.11  47.84  10.32  29.08            4   
59048  1996  34.0  0.236  297.10  33.63  77.93  55.78            4   

           region_name  
0            Вінницька  
1            Вінницька  
2            Вінницька  
3            Вінницька  
4            Вінницька  
...      

Реалізувати процедури для формування вибірок наступного виду
(включаючи елементи аналізу):
- Ряд VHI для області за вказаний рік;
- Пошук екстремумів (min та max) для вказаних областей та років,
середнього, медіани;
- Ряд VHI за вказаний діапазон років для вказаних областей;
- Для всього набору даних виявити роки, протягом яких екстремальні
посухи торкнулися більше вказаного відсотка областей по Україні (20%
областей - 5 областей з 25). Повернути роки, назви областей з
екстремальними посухами та значення VHI;

In [64]:
#Ряд VHI для області за вказаний рік
def vhi_by_region_and_year(df, region_name, year):
    result = df[(df['region_name'] == region_name) & (df['year'] == year)][['year', 'VHI']]
    return result

vhi_data = vhi_by_region_and_year(df, "Вінницька", '2013')
print(vhi_data)

     year    VHI
137  2013  42.01
138  2013  49.01
139  2013  54.76
140  2013  57.43
141  2013  57.59
142  2013  59.40
143  2013  60.13
144  2013  60.46
145  2013  61.08
146  2013  58.95
147  2013  58.13
148  2013  59.06
149  2013  61.87
150  2013  63.09
151  2013  52.05
152  2013  63.58
153  2013  62.04
154  2013  60.82
155  2013  61.14
156  2013  59.56
157  2013  55.32
158  2013  51.70
159  2013  49.54
160  2013  46.67
161  2013  43.64
162  2013  41.93
163  2013  42.26
164  2013  45.42
165  2013  49.49
166  2013  50.42
167  2013  63.41
168  2013  51.39
169  2013  33.69
185  2013  36.34
186  2013  32.92
195  2013  48.24
196  2013  48.82
197  2013  49.35
198  2013  48.23
199  2013  45.64
200  2013  41.57
202  2013  50.53
203  2013  51.36
204  2013  49.93
238  2013  47.56
240  2013  47.93
241  2013  50.45
242  2013  53.10
243  2013  55.40
244  2013  55.61
245  2013  53.08
246  2013  51.73


In [67]:
#Пошук екстремумів (min та max) для вказаних областей та років, середнього, медіани
def extremes_and_stats(df, regions, years):
    filtered_df = df[(df['region_name'].isin(regions)) & (df['year'].isin(years))]
    
    min_vhi = filtered_df['VHI'].min()
    max_vhi = filtered_df['VHI'].max()
    mean_vhi = filtered_df['VHI'].mean()
    median_vhi = filtered_df['VHI'].median()
    
    return {
        'min': min_vhi,
        'max': max_vhi,
        'mean': mean_vhi,
        'median': median_vhi
    }

stats = extremes_and_stats(df, ["Вінницька", "Волинська"], ['2020', '2021'])
print(stats)

{'min': 33.55, 'max': 71.59, 'mean': 51.23586538461538, 'median': 50.23}


In [ ]:
#Ряд VHI за вказаний діапазон років для вказаних областей:
def vhi_by_range_of_years(df, region_names, start_year, end_year):
    result = df[(df['region_name'].isin(region_names)) & (df['year'] >= start_year) & (df['year'] <= end_year)][['year', 'region_name', 'VHI']]
    return result

vhi_range = vhi_by_range_of_years(df, ["Вінницька", "Волинська"], '2015', '2020')
print(vhi_range)

      year region_name    VHI
547   2015   Вінницька  42.01
550   2019   Вінницька  29.06
615   2020   Вінницька  46.08
616   2020   Вінницька  44.74
617   2020   Вінницька  40.14
...    ...         ...    ...
3275  2018   Волинська  50.04
3276  2017   Волинська  47.22
3277  2018   Волинська  48.51
3278  2017   Волинська  43.25
3279  2018   Волинська  50.15

[624 rows x 3 columns]


In [74]:
#Для всього набору даних виявити роки, протягом яких екстремальні посухи торкнулися більше вказаного відсотка областей:
def extreme_drought_years(df, threshold_percentage=20):

    total_regions = df['region_name'].nunique()
    threshold = (threshold_percentage / 100) * total_regions
    extreme_droughts = df[df['VHI'] < 15]
    droughts_by_year = extreme_droughts.groupby('year')['region_name'].nunique()
    extreme_years = droughts_by_year[droughts_by_year >= threshold].index.tolist()
    extreme_drought_data = extreme_droughts[extreme_droughts['year'].isin(extreme_years)][['year', 'region_name', 'VHI']]
    
    return extreme_years, extreme_drought_data

extreme_years, extreme_droughts = extreme_drought_years(df, threshold_percentage=20)
print(*extreme_years)
print('_'*50)
print(extreme_droughts)

2000
__________________________________________________
       year  region_name    VHI
1643   2000    Вінницька  12.91
1657   2000    Вінницька  11.38
1675   2000    Вінницька  11.25
1690   2000    Вінницька  12.26
1692   2000    Вінницька  11.28
1765   2000    Вінницька  14.20
19179  2000     Київська  12.51
19181  2000     Київська  10.60
19182  2000     Київська  11.20
19183  2000     Київська  12.32
19184  2000     Київська  14.65
21339  2000         Київ  12.28
21360  2000         Київ  10.94
21361  2000         Київ   9.25
21362  2000         Київ   7.56
21363  2000         Київ   6.71
21364  2000         Київ   6.58
21365  2000         Київ   6.49
21366  2000         Київ   7.81
21367  2000         Київ  12.76
21368  2000         Київ  14.89
41732  2000   Харківська   9.45
41835  2000   Харківська  14.29
41951  2000   Харківська  11.45
41962  2000   Харківська  11.33
41974  2000   Харківська   9.73
42060  2000   Харківська  14.61
42088  2000   Харківська   9.36
49797  2000    Ч